In [ ]:
# 数据处理
import pandas as pd

In [ ]:
df = pd.read_csv("train.csv", index_col=0)
df.head()

In [ ]:
df['Sex'].loc[df['Sex'] == 'male']= 1
df['Sex'].loc[df['Sex'] == 'female']= 0
df

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].mean())
df 

In [ ]:
df2 = df[['Survived','Pclass','Sex','Age','SibSp','Parch']]
df2.index.astype(int)

In [ ]:
df2.to_csv("train1.csv")

In [ ]:
test_df = pd.read_csv("test.csv", index_col=0)
test_df['Sex'].loc[test_df['Sex'] == 'male']= 1
test_df['Sex'].loc[test_df['Sex'] == 'female']= 0
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
df2 = test_df[['Pclass','Sex','Age','SibSp','Parch']]
df2.to_csv("test1.csv")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class TitanicDataset(Dataset):
    def __init__(self, filepath) -> None:
        temp = np.loadtxt(filepath, delimiter=',', dtype=np.float32, skiprows=1)
        self.length = temp.shape[0]
        self.x_data = temp[:,[2,3,4,5,6]] 
        self.y_data = temp[:,[1]]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.length


train_dataset = TitanicDataset('train1.csv')
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

class Model(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear1 = torch.nn.Linear(5,3)
        self.linear2 = torch.nn.Linear(3,1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        return x


model = Model()
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)



for epoch in range(100000):
    for i,data in enumerate(train_loader, 0):
        inputs, labels = data
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        print(epoch, i, loss.item())

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()


In [ ]:
class TitanicTestDataset(Dataset):
    def __init__(self, filepath) -> None:
        temp = np.loadtxt(filepath, delimiter=',', dtype=np.float32, skiprows=1)
        self.length = temp.shape[0]
        self.x_data = temp[:,[1,2,3,4,5]] 

    def __getitem__(self, index):
        return self.x_data[index]

    def __len__(self):
        return self.length


test_dataset = TitanicTestDataset('test1.csv')
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
result_list = []

for data in test_loader:
    y = model(data)
    result_list.append(y.item())


for index in range(len(result_list)):
    if result_list[index] > 0.5:
        result_list[index] = 1
    else:
        result_list[index] = 0

In [ ]:
result = pd.read_csv('test1.csv', index_col=0)
result['Survived'] = result_list
result.head()
result2 = result['Survived']
result2
result2.to_csv('test2.csv')